In [1]:
# %load_ext blackcellmagic

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import os

In [3]:
# convert value to a rounded string with preceding $
def dollars(x):
    return "$" + str(round(x, 2))


# convert value to a rounded string with a following %
def percent(x):
    return str(round(x * 100)) + "%"

In [4]:
# create age bins
bin_names = [" <10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
bins = [0, 10, 15, 20, 25, 30, 35, 40, 1000]

In [5]:
# File to Load (Remember to Change These)
path = os.path.join("Resources", "purchase_data.csv")

# Read Purchasing File and store into Pandas data frame
df = pd.read_csv(path)
df = df.assign(
    age_range=lambda x: pd.cut(df["Age"], bins=bins, right=False, labels=bin_names)
).rename(columns={"age_range": "Age Ranges"})
df.head()

Purchase ID             SN  Age Gender  Item ID  \
0            0        Lisim78   20   Male      108   
1            1    Lisovynya38   40   Male      143   
2            2     Ithergue48   24   Male       92   
3            3  Chamassasya86   24   Male      100   
4            4      Iskosia90   23   Male      131   

                                   Item Name  Price Age Ranges  
0  Extraction, Quickblade Of Trembling Hands   3.53      20-24  
1                          Frenzied Scimitar   1.56        40+  
2                               Final Critic   4.88      20-24  
3                                Blindscythe   3.27      20-24  
4                                       Fury   1.44      20-24

In [6]:
people_df = (
    df.groupby(["SN", "Gender", "Age"])
    .agg({"Purchase ID": "count", "Price": "sum"})
    .reset_index(["Gender", "Age"])
    .rename(columns={"Purchase ID": "Purchase Count", "Price": "Total Purchase Value"})
)

people_df["Age Ranges"] = pd.cut(
    people_df["Age"], bins=bins, right=False, labels=bin_names
).astype(str)

## Player Count

* Display the total number of players


In [7]:
count = pd.DataFrame({"Total Players": [len(people_df)]})
count

Total Players
0            576

## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [8]:
purchase = pd.DataFrame(
    {
        "Number of Unique Items": [len(df["Item Name"].unique())],
        "Average Price": [dollars(df["Price"].mean())],
        "Number of Purchases": [df["Purchase ID"].count()],
        "Total Revenue": [dollars(df["Price"].sum())],
    }
)
purchase

Number of Unique Items Average Price  Number of Purchases Total Revenue
0                     179         $3.05                  780      $2379.77

## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [9]:
gender = people_df.groupby("Gender").agg(
    {"Age": "count", "Purchase Count": "sum", "Total Purchase Value": ["sum", "mean"]}
)

gender.columns = gender.columns.map("_".join)

gender = gender.sort_values("Age_count", ascending=False).rename(
    columns={
        "Age_count": "Total Counts",
        "Purchase Count_sum": "Purchase Count",
        "Total Purchase Value_sum": "Total Purchase Value",
        "Total Purchase Value_mean": "Average Purchase Price",
    }
)

gender["Percentage of Players"] = (gender["Total Counts"] / len(people_df)).apply(
    percent
)

gender[["Total Counts", "Percentage of Players"]]

Total Counts Percentage of Players
Gender                                                   
Male                            484                   84%
Female                           81                   14%
Other / Non-Disclosed            11                    2%


## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [10]:
gender["Avg Total Purchase per Person"] = (
    gender["Total Purchase Value"] / gender["Purchase Count"]
).apply(dollars)

gender["Average Purchase Price"] = gender["Average Purchase Price"].apply(dollars)

gender[["Purchase Count", "Average Purchase Price", "Avg Total Purchase per Person"]]

Purchase Count Average Purchase Price  \
Gender                                                         
Male                              652                  $4.07   
Female                            113                  $4.47   
Other / Non-Disclosed              15                  $4.56   

                      Avg Total Purchase per Person  
Gender                                               
Male                                          $3.02  
Female                                         $3.2  
Other / Non-Disclosed                         $3.35

## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [11]:
ages = people_df.groupby(["Age Ranges"]).agg(
    {"Age": "count", "Purchase Count": "sum", "Total Purchase Value": ["mean", "sum"]}
)
ages.columns = ages.columns.map("_".join)

ages = ages.rename(
    columns={
        "Age_count": "Total Count",
        "Purchase Count_sum": "Purchase Count",
        "Total Purchase Value_mean": "Average Purchase Price",
        "Total Purchase Value_sum": "Total Purchase Value",
    }
)

ages["Percentage of Players"] = ages["Total Count"].apply(
    lambda x: percent(x / ages["Total Count"].sum())
)
ages[["Total Count", "Percentage of Players"]]

Total Count Percentage of Players
Age Ranges                                   
 <10                 17                  3.0%
10-14                22                  4.0%
15-19               107                 19.0%
20-24               258                 45.0%
25-29                77                 13.0%
30-34                52                  9.0%
35-39                31                  5.0%
40+                  12                  2.0%

## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [12]:
ages["Avg Total Purchase per Person"] = (
    ages["Total Purchase Value"] / ages["Total Count"]
)

convert = [
    "Average Purchase Price",
    "Total Purchase Value",
    "Avg Total Purchase per Person",
]
for e in convert:
    ages[e] = ages[e].apply(dollars)

ages[
    [
        "Purchase Count",
        "Average Purchase Price",
        "Total Purchase Value",
        "Avg Total Purchase per Person",
    ]
]

Purchase Count Average Purchase Price Total Purchase Value  \
Age Ranges                                                               
 <10                    23                  $4.54               $77.13   
10-14                   28                  $3.76               $82.78   
15-19                  136                  $3.86              $412.89   
20-24                  365                  $4.32             $1114.06   
25-29                  101                  $3.81               $293.0   
30-34                   73                  $4.12               $214.0   
35-39                   41                  $4.76              $147.67   
40+                     13                  $3.19               $38.24   

           Avg Total Purchase per Person  
Age Ranges                                
 <10                               $4.54  
10-14                              $3.76  
15-19                              $3.86  
20-24                              $4.32  
25-29                              $3.81  
30-34                              $4.12  
35-39                              $4.76  
40+                                $3.19

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [13]:
people_df["Average Purchase Price"] = (
    people_df["Total Purchase Value"] / people_df["Purchase Count"]
)

convert = ["Average Purchase Price", "Total Purchase Value"]
for e in convert:
    people_df[e] = people_df[e].apply(dollars)

people_df.sort_values("Purchase Count", ascending=False).head()[
    ["Purchase Count", "Average Purchase Price", "Total Purchase Value"]
]

Purchase Count Average Purchase Price Total Purchase Value
SN                                                                     
Lisosia93                 5                  $3.79               $18.96
Iral74                    4                   $3.4               $13.62
Idastidru52               4                  $3.86               $15.45
Asur53                    3                  $2.48                $7.44
Inguron55                 3                   $3.7               $11.11

## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [14]:
item_df = (
    df.groupby(["Item ID", "Item Name", "Price"])
    .agg({"Purchase ID": "count"})
    .reset_index(["Price"])
    .reindex(columns=["Purchase ID", "Price"])
    .rename(columns={"Purchase ID": "Purchase Count", "Price": "Item Price"})
)
item_df["Total Purchase Value"] = item_df["Purchase Count"] * item_df["Item Price"]

item_df.sort_values("Purchase Count", ascending=False).head()

Purchase Count  \
Item ID Item Name                                                      
178     Oathbreaker, Last Hope of the Breaking Storm              12   
145     Fiery Glass Crusader                                       9   
108     Extraction, Quickblade Of Trembling Hands                  9   
82      Nirvana                                                    9   
19      Pursuit, Cudgel of Necromancy                              8   

                                                      Item Price  \
Item ID Item Name                                                  
178     Oathbreaker, Last Hope of the Breaking Storm        4.23   
145     Fiery Glass Crusader                                4.58   
108     Extraction, Quickblade Of Trembling Hands           3.53   
82      Nirvana                                             4.90   
19      Pursuit, Cudgel of Necromancy                       1.02   

                                                      Total Purchase Value  
Item ID Item Name                                                           
178     Oathbreaker, Last Hope of the Breaking Storm                 50.76  
145     Fiery Glass Crusader                                         41.22  
108     Extraction, Quickblade Of Trembling Hands                    31.77  
82      Nirvana                                                      44.10  
19      Pursuit, Cudgel of Necromancy                                 8.16

## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [15]:
item_df.sort_values("Total Purchase Value", ascending=False).head()

Purchase Count  \
Item ID Item Name                                                      
178     Oathbreaker, Last Hope of the Breaking Storm              12   
82      Nirvana                                                    9   
145     Fiery Glass Crusader                                       9   
92      Final Critic                                               8   
103     Singed Scalpel                                             8   

                                                      Item Price  \
Item ID Item Name                                                  
178     Oathbreaker, Last Hope of the Breaking Storm        4.23   
82      Nirvana                                             4.90   
145     Fiery Glass Crusader                                4.58   
92      Final Critic                                        4.88   
103     Singed Scalpel                                      4.35   

                                                      Total Purchase Value  
Item ID Item Name                                                           
178     Oathbreaker, Last Hope of the Breaking Storm                 50.76  
82      Nirvana                                                      44.10  
145     Fiery Glass Crusader                                         41.22  
92      Final Critic                                                 39.04  
103     Singed Scalpel                                               34.80